In [1]:
# %pip install statsmodels
import pandas as pd
import yfinance as yf
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta

In [3]:
# Read the CSV file
ff_factors = pd.read_csv('D:/Temp/FF6Factor.csv')

# Convert the Date column to datetime type
ff_factors['Date'] = pd.to_datetime(ff_factors['Date'])

# Function to download stock data from Yahoo Finance
def get_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data['Return'] = stock_data['Adj Close'].pct_change()  # Calculate daily returns
    stock_data = stock_data.dropna()  # Remove NaN values
    return stock_data

# Example ticker and date
ticker = 'SPY'
recommendation_date_str = '2024-01-01'
recommendation_date = datetime.strptime(recommendation_date_str, '%Y-%m-%d') 
end_date = recommendation_date + timedelta(days = 100)

# Calculate the start date based on the end date (272 to 21 trading days before the given date)
start_date = end_date - timedelta(days=500)  # Approximately one year (252 trading days)
stock_data = get_stock_data(ticker, start_date, end_date)

# Select the appropriate time window
filtered_stock_data = stock_data.loc[(stock_data.index >= recommendation_date - timedelta(days=396)) & 
                                     (stock_data.index <= recommendation_date - timedelta(days=30))]

# Merge the stock data with the factor data based on the date
merged_data = pd.merge(filtered_stock_data, ff_factors, left_index=True, right_on='Date')

# Subtract the risk-free rate (RF) from the stock returns to get excess returns
merged_data['Excess Return'] = merged_data['Return'] * 100 - merged_data['RF']

# Select the independent variables (factors)
X = merged_data[['MKT-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM']]
X = sm.add_constant(X)  # Add a constant (intercept) to the model

# The dependent variable (excess return)
y = merged_data['Excess Return']

# Create and fit the regression model
model = sm.OLS(y, X).fit()

# Display the regression results
print(model.summary())

# Define the dates for the 2 and 3-month periods
start_future = recommendation_date
end_future_2m = start_future + timedelta(days=61)  # Approx. 2 months
end_future_3m = start_future + timedelta(days=91)  # Approx. 3 months

# Instead of downloading the data again, filter the already downloaded data
future_data = stock_data.loc[(stock_data.index >= start_future) & 
                             (stock_data.index <= end_future_3m)].copy()

# Calculate the factor returns for the period
future_factors = ff_factors.loc[(ff_factors['Date'] >= start_future) & 
                            (ff_factors['Date'] <= end_future_3m)].copy()

# Set 'Date' column as index for both DataFrames
future_data.index = future_data.index.normalize()  # Normalize to just the date part
future_factors.set_index('Date', inplace=True)

# Align indices and calculate excess return daily for future periods
future_data['Excess Return'] = future_data['Return'].sub(future_factors['RF'] / 100, fill_value=0)

# Calculate the cumulative excess return for the 2 and 3-month periods
future_data['Cumulative Return'] = (1 + future_data['Excess Return']).cumprod() - 1

# Calculate the predicted returns (alpha) using the model for the 2 and 3-month periods
predicted_return_2m = model.predict(sm.add_constant(future_factors[['MKT-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM']][:len(future_data.loc[:end_future_2m])]))
predicted_return_3m = model.predict(sm.add_constant(future_factors[['MKT-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM']][:len(future_data.loc[:end_future_3m])]))

# Correct calculation for 6-factor alpha by cumulative returns
cumulative_predicted_return_2m = (1 + predicted_return_2m / 100).cumprod() - 1
cumulative_predicted_return_3m = (1 + predicted_return_3m / 100).cumprod() - 1

# Calculate the 6-factor alpha for 2 and 3-month periods
alpha_2m = future_data.loc[:end_future_2m, 'Cumulative Return'].iloc[-1] * 100 - cumulative_predicted_return_2m.iloc[-1] * 100
alpha_3m = future_data.loc[:end_future_3m, 'Cumulative Return'].iloc[-1] * 100 - cumulative_predicted_return_3m.iloc[-1] * 100

print(f"6-factor alpha for 2-month period: {alpha_2m}")
print(f"6-factor alpha for 3-month period: {alpha_3m}")


[*********************100%%**********************]  1 of 1 completed
                            OLS Regression Results                            
Dep. Variable:          Excess Return   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 1.147e+04
Date:                Thu, 29 Aug 2024   Prob (F-statistic):          5.36e-297
Time:                        12:40:10   Log-Likelihood:                 388.69
No. Observations:                 252   AIC:                            -763.4
Df Residuals:                     245   BIC:                            -738.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

In [ ]:
import os

# Ensure the directory exists
output_directory = r'D:/Temp/FF'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Reset index to save 'Date' as a column
merged_data.reset_index(inplace=True)
future_data.reset_index(inplace=True)
future_factors.reset_index(inplace=True)

# Save the merged data with excess return
merged_data.to_csv(os.path.join(output_directory, 'merged_data.csv'), index=False)

# Save the future data with cumulative returns
future_data.to_csv(os.path.join(output_directory, 'future_data.csv'), index=False)

# Save the factor data for the future periods
future_factors.to_csv(os.path.join(output_directory, 'future_factors.csv'), index=False)

# Make sure the index is properly aligned and get the correct dates
dates = future_factors['Date'].iloc[:len(future_data)]

# Save the predicted returns
predicted_returns_df = pd.DataFrame({
    'Date': dates,
    'Predicted Return 2M': predicted_return_2m,
    'Predicted Return 3M': predicted_return_3m
})
predicted_returns_df.to_csv(os.path.join(output_directory, 'predicted_returns.csv'), index=False)

# Optionally save the model summary to a text file for review
with open(os.path.join(output_directory, 'model_summary.txt'), 'w') as f:
    f.write(model.summary().as_text())

print("All data has been saved to the D:/Temp/FF directory.")
